### On top of the Embeddedings gained from Bertweet, I added simple Neural network to 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset, DataLoader
import torchvision
import torchvision.transforms as transforms

from __Preprocess import *

Error: Kernel is dead

In [13]:
data = pd.read_csv('./data/_PHEME_text_AVGw2v.csv')
y = pd.read_csv('./data/_PHEME_target.csv')
events = pd.read_csv('./data/_PHEME_text.csv').Event

test_data = pd.read_csv('./data/_RHI_text_AVGw2v.csv')
test_y = pd.read_csv('./data/_RHI_target.csv')

In [14]:
events.head()

0    charliehebdo
1    charliehebdo
2    charliehebdo
3    charliehebdo
4    charliehebdo
Name: Event, dtype: object

In [4]:
test_data.drop(['token'],axis=1,inplace=True)

In [5]:
# for dataset in [train_avg, test_avg, train_doc, test_doc, valid_avg, valid_doc, df_bertweet, df_valid_bertweet]:
#     dataset.drop(['text','text_token','urls', 'urls_expanded','user_url'], axis=1, inplace=True)

### Data with BERTweet Embedding consists of 797 Dimensions: which are 767 Embeddings and 30 additional features

### Other Dataset consists of 328 dimensions

### Print to Test

In [5]:
print("Shape of the Train and test data of Averaged Word2Vec: {}/{}".format(data.shape, test_data.shape))
# print("Shape of the Train and test data of Doc2vec: {}/{}".format(train_doc.shape, test_doc.shape))
# print("\nShape of the validation data of Avg: {}".format(valid_avg.shape))
# print("Shape of the validation data of Doc2vec: {}".format(valid_doc.shape))
# print("\nShape of the data w/ BERTweet: {}".format(df_bertweet.shape))
# print("\nShape of the validation data w/ BERTweet: {}".format(df_valid_bertweet.shape))

Shape of the Train and test data of Averaged Word2Vec: (5802, 200)/(5227, 200)


In [6]:
# Dropping Target values from the dataset
# train_y = train_avg.isRumor
# test_y = test_avg.isRumor
# valid_y = valid_avg.isRumor
# df_bertweet_y = df_bertweet.isRumor
# df_valid_bertweet_y = df_valid_bertweet.isRumor
# for dataset in [train_avg, test_avg, train_doc, test_doc, valid_avg, valid_doc, df_bertweet, df_valid_bertweet]:
#     dataset.drop(['isRumor'], axis=1, inplace=True)

# print(train.shape, test_y.shape, valid_y.shape, df_bertweet_y.shape, df_valid_bertweet_y.shape)
print(y.shape, test_y.shape)

(5802, 1) (5227, 1)


In [7]:
# # Base dataset are the baseline feature set to be inputted to the model
# # Here, 4 features are dropped for their lack of predictive power

# train_avg_base = train_avg.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# test_avg_base = test_avg.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# valid_avg_base = valid_avg.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# train_doc_base = train_doc.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# test_doc_base = test_doc.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# valid_doc_base = valid_doc.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# bertweet_base = df_bertweet.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# bertweet_valid_base = df_valid_bertweet.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)

# Torch

In [5]:
# 위의 데이터들을 torchTensor로 변환한뒤 Unsqueeze한다.
# 이후 TensorDataset를 생성한다. (X, y 값을 담은 텐서들을 인자로 넘겨줌)

# tensor_x = torch.Tensor(np.array(data))
# tensor_y = torch.Tensor(np.array(y))
tensor_x = torch.tensor(data.values)
tensor_y = torch.tensor(y.values)

tensor_x = tensor_x.unsqueeze(1)
tensor_y = tensor_y.unsqueeze(1)

data_dataset = TensorDataset(tensor_x,tensor_y)

In [ ]:
# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train.values)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
3


In [31]:
# 훈련 데이터 / 테스트 데이터를 torch.utils.data.random_split()를 통해서 나눠준다
input_len = len(task1_dataset)
test_ratio = 0.1
test_size = int(input_len * test_ratio)
train_size = input_len - test_size

print("Length of the Inputs are: ",input_len, train_size, test_size)

train_data, test_data = torch.utils.data.random_split(task1_dataset, (train_size, test_size))
print(len(train_data), len(test_data))

Length of the Inputs are:  5802 5222 580
5222 580


In [32]:
# 위에서 생성한 훈련/테스트 데이터를 각각 DataLoader를 호출해 데이터 로더를 생성한다.
# 참고로 이 코드에서는 task1_dataset -> tensor_x/y -> train_avg_base/train_y를 사용하고 있다.
task1_train_dataloader = DataLoader(train_data, batch_size=6, shuffle=True, num_workers=2)
task1_test_dataloader = DataLoader(test_data, batch_size=6, shuffle=True, num_workers=2)

In [38]:
# FC_net을 생성 -> 
class FC_net(nn.Module):
    def __init__(self):
        super(FC_net, self).__init__() # 1*20
        self.fc1 = nn.Linear(792, 130) # 420
        self.fc2 = nn.Linear(130, 60)
        self.fc3 = nn.Linear(60, 1)

        self.drop_2 = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x
        
task1_model = FC_net()

In [39]:
print(task1_model.parameters)

<bound method Module.parameters of FC_net(
  (fc1): Linear(in_features=792, out_features=130, bias=True)
  (fc2): Linear(in_features=130, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=1, bias=True)
  (drop_2): Dropout(p=0.2, inplace=False)
)>


In [40]:
import torch.optim as optim

criterion = nn.BCELoss()
# optimizer = optim.SGD(task1_model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(task1_model.parameters(), lr=0.01)

In [41]:
train_loss = []
train_accuracy = []

prev_loss = 10
PATH = "./state_dict_BERT_fc.pt"
best_acc = 10.0
num_epochs = 10

val_corrects_list = []
val_loss_list = []

In [42]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    running_loss = 0.0
    running_corrects = 0
    task1_model.train()  # Set model to training mode
    for i, data in enumerate(task1_train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # inputs, labels = inputs.float(), labels.long()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = task1_model(inputs)

        labels = labels.unsqueeze(1).float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()    

        running_loss += loss.item()
        running_corrects += torch.sum(outputs == labels.data)
        # print(running_corrects)

    epoch_loss = running_loss / train_size
    epoch_acc = running_corrects.double() / train_size
    train_loss.append(epoch_loss)
    train_accuracy.append(epoch_acc)

    print('Train) Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    # if epoch_loss < best_acc:
    #     # print("prev_loss: {:.5f}".format(prev_loss))
    #     # print("loss: {:.5f}".format(loss))
    #     print("Saving the best model w/ loss {:.4f}".format(epoch_loss))
    #     torch.save(task1_model.state_dict(),PATH)
    #     best_acc = epoch_loss

Epoch 0/9
----------
Train) Loss: 0.3190 Acc: 0.0042
Epoch 1/9
----------
Train) Loss: 5.4705 Acc: 0.6325
Epoch 2/9
----------
Train) Loss: 5.7149 Acc: 0.6568
Epoch 3/9
----------
Train) Loss: 5.7213 Acc: 0.6568
Epoch 4/9
----------
Train) Loss: 5.7149 Acc: 0.6568
Epoch 5/9
----------
Train) Loss: 5.7213 Acc: 0.6568
Epoch 6/9
----------
Train) Loss: 5.7149 Acc: 0.6568
Epoch 7/9
----------
Train) Loss: 5.7149 Acc: 0.6568
Epoch 8/9
----------
Train) Loss: 5.7213 Acc: 0.6568
Epoch 9/9
----------
Train) Loss: 5.7213 Acc: 0.6568


In [43]:
task1_model.eval()
correct = 0
total = 0
test_loss = 0
outputs_list = []
y_list = []

with torch.no_grad():
    val_loss = 0

    for i, data in enumerate(task1_test_dataloader):
        x, y = data
        x, y = x.float(), y.long()
        outputs = task1_model(x)
        loss = criterion(outputs, y.unsqueeze(1).float())
        _, predicted = torch.max(outputs.data, 1)
        outputs_list.append(predicted[:])
        total += y.size(0)
        correct += (predicted == y).double().sum().item()
        val_loss += loss.item()
        y_list.append(y)

print('Accuracy of the test dataset is: %d %%' % (100 * correct / total))
print("Loss of validation set: {:.5f}".format((val_loss / test_size)))
acc = (100 * correct / total)

Accuracy of the test dataset is: 68 %
Loss of validation set: 5.20883
